In [1]:
import numpy as np
import random

num_entities = 10
num_validators = 500000
num_slots = 300000
fixed_reward = 0.025
sigma_mev_reward = 1
split_threshold = 32

# Define the mean MEV rewards for each entity
entity_mev_mean_rewards = [0.5] + [0.3]*2 + [0.1]*(num_entities-3)

# Create validator_entity_mapping and initialize rewards
validator_entity_mapping = [i % num_entities for i in range(num_validators)]
validator_rewards = np.zeros(num_validators)
entity_rewards = np.zeros(num_entities)

# Create lists of validators per entity
validators_per_entity = [[] for _ in range(num_entities)]
for validator_idx, entity in enumerate(validator_entity_mapping):
    validators_per_entity[entity].append(validator_idx)

# Run the simulation
for _ in range(num_slots):
    # Select a random validator
    chosen_validator = random.randrange(num_validators)

    # Assign rewards
    block_reward = fixed_reward
    chosen_entity = validator_entity_mapping[chosen_validator]
    mev_reward_mean = entity_mev_mean_rewards[chosen_entity]
    mev_reward = np.random.lognormal(mean=np.log(mev_reward_mean), sigma=sigma_mev_reward)
    total_reward = block_reward + mev_reward

    validator_rewards[chosen_validator] += total_reward
    entity_rewards[chosen_entity] += total_reward

    # Check if we need to spawn a new validator for any entity
    for entity_idx in range(num_entities):
        if entity_rewards[entity_idx] >= split_threshold:
            # Spawn a new validator for this entity
            num_validators += 1
            validator_entity_mapping.append(entity_idx)
            validator_rewards = np.append(validator_rewards, 0)
            validators_per_entity[entity_idx].append(num_validators - 1)

            # Deduct the rewards used to spawn the new validator from the existing validators
            to_deduct = split_threshold
            for validator_idx in validators_per_entity[entity_idx]:
                if to_deduct > 0:
                    deduct_amount = min(validator_rewards[validator_idx], to_deduct)
                    validator_rewards[validator_idx] -= deduct_amount
                    entity_rewards[entity_idx] -= deduct_amount
                    to_deduct -= deduct_amount

# Print the top and bottom 20 validator stakes
sorted_indices = np.argsort(validator_rewards)
print("Top 20 validator stakes:")
for idx in sorted_indices[-20:]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

print("Bottom 20 validator stakes:")
for idx in sorted_indices[:20]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

# Count validators per entity
validators_count_per_entity = [len(validators) for validators in validators_per_entity]

# Calculate the total value each entity controls
entity_values = np.array(validators_count_per_entity) * 32 + entity_rewards

# Print the results
print("Validators per entity:")
for i in range(num_entities):
    print(f"Entity {i}: {validators_count_per_entity[i]}")

print("Total value each entity controls:")
for i in range(num_entities):
    print(f"Entity {i}: {entity_values[i]}")


Top 20 validator stakes:
Validator 279464: 1.125592409324459
Validator 311300: 1.1302410352561985
Validator 209540: 1.1444369015521103
Validator 277711: 1.1461497482015583
Validator 225470: 1.1578721077745278
Validator 240359: 1.2182531382734991
Validator 502688: 1.2350101726001401
Validator 369152: 1.236167821562591
Validator 234760: 1.2662809547270635
Validator 293410: 1.3127261621551498
Validator 404697: 1.3621015396180702
Validator 367090: 1.4047131600536948
Validator 109010: 1.4918957262140535
Validator 461812: 1.5002778453880512
Validator 431900: 1.8016300986998073
Validator 16201: 1.9942770270405294
Validator 187122: 2.6074304365277996
Validator 425669: 2.9233921708093678
Validator 289391: 5.174753482876701
Validator 419370: 5.399035100120041
Bottom 20 validator stakes:
Validator 0: 0.0
Validator 335234: 0.0
Validator 335233: 0.0
Validator 335232: 0.0
Validator 335231: 0.0
Validator 335230: 0.0
Validator 335229: 0.0
Validator 335228: 0.0
Validator 335227: 0.0
Validator 335226: 0